In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
tf.config.list_physical_devices('GPU')
tf.config.list_physical_devices('CPU')

In [ ]:
df = pd.read_csv('stock.csv',index_col='date', parse_dates=True)
df.info()
df.head()

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(df['close'], label='Close Price')
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(df['volume'], label='Volume')
plt.show()

In [ ]:
"""收盘价的概率密度分布图"""
sns.kdeplot(df['close'], fill=True)
"""
多峰分布
右偏分布 右侧稀疏，尾部延展到较高的价格区间
"""

In [ ]:
"""趋势分析"""
window_size =30
df['ma_trend'] = df['close'].rolling(window=window_size).mean()
plt.figure(figsize=(12,6))
plt.plot(df['close'],label='Original')
df['ma_trend'].plot(label=f'{window_size}-Day Moving Average')
plt.legend()
plt.title('Trend Analysis with Moving Average')
plt.show()

In [ ]:
result = sm.tsa.seasonal_decompose(df['close'], model='additive', period=96) 
# 可视化分解结果
plt.figure(figsize=(24,9))
result.plot()
plt.tight_layout()
plt.show()
"""
残差分析：
2021-2023年，残差分析波动明显
"""

In [ ]:
df_2021_2023 = df[df.index.year.isin([2021, 2022])]
df_2021_2023.head()